In [ ]:
import os
from database_tools.tools import BuildDatabase

repo_dir = '/media/cam/CAPSTONEDB/database_tools/'
os.chdir(repo_dir)

config = dict(
    low=0.5,
    high=8.0,
    sim1=0.7,
    sim2=0.9,
    snr_t=22.5,
    hr_diff=1/6,
    f0_low=0.667,
    f0_high=3.0,
    abp_min_bounds=[40, 100],  # this is largest possible range of valid bp values
    abp_max_bounds=[70, 190],  # ^
)

worker = BuildDatabase(
    output_dir='data-2022-09-29/',
    config=config,
    win_len=256,
    fs=125,
    samples_per_file=2500,
    samples_per_patient=600,
    max_samples=300000,
    data_dir='physionet.org/files/mimic3wdb/1.0/',
)

df = worker.run()

In [ ]:
import numpy as np

low=0.5
high=8.0
sim1=0.7
sim2=0.9
snr_t=22.5
hr_diff=1/6
f0_low=0.667
f0_high=3.0
abp_min_bounds=[40, 100]
abp_max_bounds=[70, 190]

time_sim = 0.917812
spec_sim = 0.982195	
snr_p = 31.567170
snr_a = 18.929158
f0_p = 124.640565 / 60
f0_a = 124.300533 / 60
f0 = np.array([f0_p, f0_a])
min_ = 72.65625
max_ = 140.62500

valid = False
if np.nan in [time_sim, spec_sim, snr_p, snr_a, f0_p, f0_a, min_, max_]:
    pass
elif (time_sim < sim1) | (spec_sim < sim1):
    pass
elif ( (snr_p < snr_t) | (snr_a < snr_t) ) & ( (time_sim < sim2) | (spec_sim < sim2) ):
    pass
elif ( np.abs(f0_p - f0_a) > hr_diff ) & ( (time_sim < sim2) | (spec_sim < sim2) ):
    pass
elif (f0 < f0_low).any() | (f0 > f0_high).any():
    pass
elif (min_ < abp_min_bounds[0]) | (max_ > abp_max_bounds[1]):
    pass
elif (max_ < abp_max_bounds[0]) | (max_ > abp_max_bounds[1]):
    pass
else:
    valid = True
valid

In [ ]:
import os
import pandas as pd
from database_tools.tools import DataEvaluator

pd.options.display.max_rows = 1000

repo_dir = '/media/cam/CAPSTONEDB/database_tools/'
os.chdir(repo_dir)

df = pd.read_csv('data-2022-09-29/mimic3_stats.csv')
worker = DataEvaluator(stats=df)

figs = worker.run()

In [ ]:
df.head(1000)

In [ ]:
import glob
import json
import numpy as np

ppg = []
abp = []

with open('../data-2022-09-29/mimic3/mimic3_0000000.jsonlines', 'r') as f:
    for sample in f:
        sample = json.loads(sample)
        ppg.append(sample['ppg'])
        abp.append(sample['abp'])


ppg = np.array(ppg).flatten()
abp = np.array(abp).flatten()

In [ ]:
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler

fig = go.FigureWidget()
fig.add_scatter()
fig.update_layout(
    xaxis_title='Time',
    yaxis_title='Amplitude',
    width=1000,
    height=800,
)
fig
# fig.data[0].y = y

In [ ]:
# for f in $files; do mongoimport --db MIMICDB --file $f; done

# from pymongo import MongoClient

# uri = 'mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+1.6.0'
# client = MongoClient(uri)

# coll = client.MIMICDB.mimic3_0000067

# cursor = coll.find()
# for doc in cursor:
#     print(doc)